In [2]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

## Scrapping sur [consoGlobe](https://www.consoglobe.com/eau-virtuelle-conflit-3120-cg) (1)

In [3]:
url_consoGlobe_1 = "https://www.consoglobe.com/eau-virtuelle-conflit-3120-cg"

In [4]:
# User-Agent is UNIQUE to every internet user
header = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36"}

In [5]:
response_consoGlobe_1 = requests.get(url_consoGlobe_1, headers = header)

In [6]:
soup_consoGlobe_1 = BeautifulSoup(response_consoGlobe_1.text, 'html.parser')

In [7]:
text = [s.get_text() for s in soup_consoGlobe_1.find_all('div', {'class': 'entry-content'})[0].find_all('li')]

In [8]:
text[1:-1]

['10 litres= 1 feuille de papier A4',
 '30 litres= 1 tasse de thé',
 '50 litres= 1 orange',
 '140 litres= 1 tasse de café',
 '200 litres= 1 oeuf',
 '1000 litres= 1 litre de lait',
 '1500 litres= 1 kg de sucre de canne',
 '2700 litres= 1 tee-shirt en coton',
 '1 kg de boeuf \xa0 15 500 litres d’eau virtuelle',
 '1 kg de fromage \xa0 5 000 litres d’eau virtuelle',
 '1 kg de coton \xa0 5 260 litres d’eau virtuelle',
 '5 804 litres d’eau sont nécessaires pour cuisiner un simple «\xa0wonton\xa0» chinois.',
 '1 kg de poulet 3 900',
 '1 chandail de coton 2 700']

In [9]:
delimiter_words = ("= ", " \xa0 ", " sont nécessaires pour cuisiner ")
regex = r"\b(?:{})\b".format("|".join(delimiter_words))
eau_virtuelle = []
produit = []
for t in text[1:-1]:
    if "litre" in t:
        if int(t.split()[0]) > 1:
            eau_virtuelle.append([r.strip().replace(' litres d’eau virtuelle', '').replace(' litres d’eau', '').replace(' litres', '').replace(' ', '') for r in re.split(regex, t)][0])
            produit.append([r.strip().replace('\xa0', ' ').replace('un', '1').replace('.', '') for r in re.split(regex, t)][1])
        else:
            eau_virtuelle.append([r.replace(' litres d’eau virtuelle', '').replace(' ', '') for r in re.split(regex, t)][1])
            produit.append([r.strip().replace('\xa0', ' ') for r in re.split(regex, t)][0])
    else:
        eau_virtuelle.append(('').join(t.split()[-2:]))
        produit.append((' ').join(t.split()[:-2]))

In [10]:
eau_virtuelle = [int(e) for e in eau_virtuelle]
eau_virtuelle

[10, 30, 50, 140, 200, 1000, 1500, 2700, 15500, 5000, 5260, 5804, 3900, 2700]

In [11]:
produit

['1 feuille de papier A4',
 '1 tasse de thé',
 '1 orange',
 '1 tasse de café',
 '1 oeuf',
 '1 litre de lait',
 '1 kg de sucre de canne',
 '1 tee-shirt en coton',
 '1 kg de boeuf',
 '1 kg de fromage',
 '1 kg de coton',
 '1 simple « wonton » chinois',
 '1 kg de poulet',
 '1 chandail de coton']

In [12]:
df_consommation_1 = pd.DataFrame(zip(produit, eau_virtuelle, ["consoglobe.com"]*len(eau_virtuelle)), columns=["Produit de consommation", "Eau virtuelle (litres)", "Source"])

In [13]:
df_consommation_1

,Produit de consommation,Eau virtuelle (litres),Source
0,1 feuille de papier A4,10,consoglobe.com
1,1 tasse de thé,30,consoglobe.com
2,1 orange,50,consoglobe.com
3,1 tasse de café,140,consoglobe.com
4,1 oeuf,200,consoglobe.com
5,1 litre de lait,1000,consoglobe.com
6,1 kg de sucre de canne,1500,consoglobe.com
7,1 tee-shirt en coton,2700,consoglobe.com
8,1 kg de boeuf,15500,consoglobe.com
9,1 kg de fromage,5000,consoglobe.com


In [14]:
file_name = '../data/data_consoGlobe1.csv'
df_consommation_1.to_csv(file_name, sep=',', 
                  encoding='utf-8', index=False)